In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
laptop = pd.read_csv("../laptop.csv")
detailed_df = pd.read_csv('detailed_df.csv')
df2 = detailed_df.copy()

: 

In [4]:
laptop['cpu cores'] = detailed_df['cpu cores']

In [5]:
# laptop['storage capacity'] = laptop['storage capacity'].apply(lambda x: int(x))
laptop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1540 non-null   object 
 1   price             1540 non-null   int64  
 2   brand             1540 non-null   object 
 3   model             1540 non-null   object 
 4   processor         1540 non-null   float64
 5   ram memory        1540 non-null   int64  
 6   display size      1540 non-null   float64
 7   storage capacity  1540 non-null   int64  
 8   cpu cores         1235 non-null   object 
 9   graphics card     0 non-null      float64
 10  graphics memory   240 non-null    object 
 11  rating            1540 non-null   float64
 12  reviews           1540 non-null   int64  
 13  name              1540 non-null   object 
 14  url               1540 non-null   object 
dtypes: float64(4), int64(4), object(7)
memory usage: 180.6+ KB


In [16]:
# laptop.loc[laptop['cpu cores'] == 'Quad Core', 'cpu cores'] = '4'
# laptop.loc[laptop['cpu cores'] == 'Octa Core', 'cpu cores'] = '8'
# # laptop.loc[laptop['cpu cores'] == 'Hexa Core', 'cpu cores'] = '6'
# # laptop.loc[laptop['cpu cores'] == 'Single Core', 'cpu cores'] = '1'
# # # laptop.loc[laptop['cpu cores'] == 'Dual Core', 'cpu cores'] = '2'
# laptop.loc[laptop['storage capacity'] == '1TB', 'storage capacity'] = '1024'
# laptop.loc[laptop['storage capacity'] == '4', 'storage capacity'] = '256'



laptop['cpu cores'].value_counts()
# laptop['cpu cores'].isnull().sum()

cpu cores
4                326
Not Specified    270
8                233
10               149
6                119
2                100
12                42
1                 27
14                 3
0                  1
Name: count, dtype: int64

In [25]:
laptop['cpu cores'].isnull().sum()

270

In [8]:
def clean_text(text):
    if isinstance(text, str):  # Check if the text is a string
        # Define the pattern to match unwanted characters
        pattern = r'[\/,.\(\):;\-]'
        # Use re.sub() to replace unwanted characters with an empty string
        cleaned_text = re.sub(pattern, '', text)
        return cleaned_text
    return text  # If the text is not a string, return it as is

In [9]:
df2['details'] = df2['details'].astype(str)
df2['details'] = df2['details'].apply(clean_text)

In [28]:
# # Clean and standardize the 'details' column
df2['details'] = df2['details'].str.replace(r'(\d+)\s+cores', r'\1cores', regex=True)
df2['details'] = df2['details'].str.replace(r'(\d+)\s+Cores', r'\1Cores', regex=True)

# Iterate over the DataFrame rows
for index, row in df2.iterrows():
    # Check for null or "Not Specified" in 'details' column
    if pd.isnull(row['details']) or row['details'] == 'Not Specified':
        continue

    # Use regex to find the number of cores
    match = re.search(r'(\d+)[-]?[cC]ore[sS]', row['details'])
    
    if match:
        cores = int(match.group(1))
        df2.loc[index, 'cpu cores'] = cores
        print("Row:", index)
        print("Cores:", df2.loc[index, 'cpu cores'])
        print("Detail:", row['details'])

# Display the updated DataFrame
print(df2)


Row: 2
Cores: 10
Detail: Brand Dell Model Vostro 3520 Processor Intel Core i5 12th Generation 1235U 12 MB cache 10cores 12 threads up to 440 GHz Turbo Memory 16GB DDR4 RAM Storage 512GB NVMe SSD Display 156" 120Hz FHD Display Battery 3Cell Battery Color Carbon Black Warranty 1Year Warranty
Row: 8
Cores: 8
Detail: Brand HP Model Victus 15FA1093DX Processor Intel Core i513420H up to 460 GHz 12 MB Cache & 8Cores RAM 16 GB DDR43200 MHz RAM Storage 512 GB PCIe NVMe M2 SSD Display 156inch diagonal IPS microedge 250 nits antiglare Ultra slim 144 Hz Narrow Border Graphics NVIDIA GeForce RTX 3050 6 GB  Camera HP True Vision 720p HD camera Network and Communication Intel® WiFi 6E AX211 2×2 and Bluetooth 53 Battery 4cell 70 Wh Liion polymer Operating System Microsoft Windows 11 Warranty 1 Year International Warranty Color Blue
Row: 16
Cores: 6
Detail: Brand Dell Model Vostro 3520 Processor Intel Core i3 12th Generation 1215U 10M Cache 6Cores 8 Threads Memory 16GB DDR4 RAM Storage 512GB NVMe SSD D

In [21]:
detailed_df = df2.copy()

In [26]:
# df2['cpu cores'] = df2['cpu cores'].apply(lambda x: int(x) if pd.notna(x) and x != 'Not Specified' else np.nan if x == 'Not Specified' else x)
df2['cpu cores'] = df2['cpu cores'].apply(lambda x: int(x) if pd.notnull(x) and x != 'Not Specified' else x)
df2['cpu cores'].value_counts()

cpu cores
4                324
Not Specified    259
8                232
10               150
6                120
2                 97
12                39
14                35
1                 27
24                 5
16                 3
5                  2
20                 1
0                  1
Name: count, dtype: int64

In [29]:
df2['cpu cores'].isnull().sum()

245

In [14]:
# columns = list(df2.columns)
# columns[0], columns[2] = columns[2], columns[0]
# df2 = df2[columns]

# df2.to_csv('detailed_df.csv', index=False)

laptop['cpu cores'] = detailed_df['cpu cores']


# laptop.info()

In [30]:

laptop.to_csv("../laptop.csv", index=False)
detailed_df.to_csv('detailed_df.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'detailed_df.csv'